In [1]:
import pandas as pd
import utils
#cerrega todo os arquivos
dfNTLNew = pd.read_csv("./data/PT/NTLHNew.tsv", sep="\t") # esse cara podemos fazer o seguinte filtro: dfNTLNew.loc[dfNTLNew["versiculo"] >= "1" ]
dfNTLOld = pd.read_csv("./data/PT/NTLHOld.tsv", sep="\t") # esse cara podemos fazer o seguinte filtro: dfNTLOld.loc[dfNTLOld["versiculo"] >= "1" ]

dfARCNew = pd.read_csv("./data/PT/ARCNew.tsv", sep="\t") # esse cara podemos fazer o seguinte filtro: dfARCNew.loc[dfARCNew["versiculo"] >= "1" ]
dfARCOld = pd.read_csv("./data/PT/ARCOld.tsv", sep="\t") # esse cara podemos fazer o seguinte filtro: dfARCOld.loc[dfARCOld["versiculo"] >= "1" ]

dfNVIPTNew = pd.read_csv("./data/PT/NVIPTNew.tsv", sep="\t") # esse cara podemos fazer o seguinte filtro: dfNVIPTNew.loc[dfNVIPTNew["versiculo"] >= "1" ] 
dfNVIPTOld = pd.read_csv("./data/PT/NVIPTOld.tsv", sep="\t") # esse cara podemos fazer o seguinte filtro: dfNVIPTOld.loc[dfNVIPTOld["versiculo"] >= "1" ] 

dfNVTNew = pd.read_csv("./data/PT/NVTNew.tsv", sep="\t") # esse cara podemos fazer o seguinte filtro: dfNVTNew.loc[dfNVTNew["versiculo"] >= "1" ] 
dfNVTOld = pd.read_csv("./data/PT/NVTOld.tsv", sep="\t") # esse cara podemos fazer o seguinte filtro: dfNVTOld.loc[dfNVTOld["versiculo"] >= "1" ] 

dfOLNew = pd.read_csv("./data/PT/OLNew.tsv", sep="\t") # esse cara podemos fazer o seguinte filtro: dfOLNew.loc[dfOLNew["versiculo"] >= "1" ]
dfOLOld = pd.read_csv("./data/PT/OLOld.tsv", sep="\t") # esse cara podemos fazer o seguinte filtro: dfOLOld.loc[dfOLOld["versiculo"] >= "1" ]

dfVFLNew = pd.read_csv("./data/PT/VFLNew.tsv", sep="\t") # esse cara podemos fazer o seguinte filtro: dfVFLNew.loc[dfVFLNew["versiculo"] >= "1" ]

#eu posso fazer esses filtros ou remover tudo o que tenha o nome do livro por extenso e o nome do estilo. Se fizer isso presaremos reajustasr os indices

In [2]:
dfNTLNew

,Unnamed: 0,estilo,livro,capitulo,versiculo,texto
0,0,NTLH,Mateus,1,1,Esta é a lista dos antepassados de Jesus Crist...
1,1,NTLH,Mateus,1,2000,Sociedade Bíblica do Brasil . Todos os direito...
2,2,NTLH,Mateus,1,2,"Abraão foi pai de Isaque , Isaque foi pai de J..."
3,3,NTLH,Mateus,1,3,"Judá foi pai de Peres e de Zera , e a mãe dele..."
4,4,NTLH,Mateus,1,4,"Arão foi pai de Aminadabe , que foi pai de Nas..."
...,...,...,...,...,...,...
7892,7892,NTLH,Apocalipse,22,17,O Espírito e a Noiva dizem : — Venha ! Aquele ...
7893,7893,NTLH,Apocalipse,22,18,"Eu , João , aviso solenemente aos que ouvem as..."
7894,7894,NTLH,Apocalipse,22,19,"E , se alguma pessoa tirar alguma coisa das pa..."
7895,7895,NTLH,Apocalipse,22,20,Aquele que dá testemunho de tudo isso diz : — ...


In [3]:
#normalizando os nomes do novo testamento

print(set(dfNTLNew["livro"]) - set(dfARCNew["livro"]))
print(set(dfNTLNew["livro"]) - set(dfVFLNew["livro"]))
print(set(dfNTLNew["livro"]) - set(dfNVIPTNew["livro"]))
print(set(dfNTLNew["livro"]) - set(dfNVTNew["livro"]))
print(set(dfNTLNew["livro"]) - set(dfOLNew["livro"]))

dfNVIPTNew["livro"] = dfNVIPTNew["livro"].replace("Filemón", "Filemom")
dfNVTNew["livro"]   = dfNVTNew["livro"].replace("Filemón", "Filemom")
dfOLNew["livro"]    = dfOLNew["livro"].replace("Filemón", "Filemom")

print(set(dfNTLNew["livro"]) - set(dfARCNew["livro"]))
print(set(dfNTLNew["livro"]) - set(dfVFLNew["livro"]))
print(set(dfNTLNew["livro"]) - set(dfNVIPTNew["livro"]))
print(set(dfNTLNew["livro"]) - set(dfNVTNew["livro"]))
print(set(dfNTLNew["livro"]) - set(dfOLNew["livro"]))


set()
set()
set()
set()
set()
set()
set()
set()
set()
set()


In [8]:
dictOldBooksNTLH = {"Gênesis": 50,
"Êxodo": 40,
"Levítico": 27,
"Números": 36,
"Deuteronômio": 34,
"Josué": 24,
"Juízes": 21,
"Rute": 4,
"1%20Samuel": 31,
"2%20Samuel": 24,
"1%20Reis": 22,
"2%20Reis": 25,
"1%20Crônicas": 29,
"2%20Crônicas": 36,
"Esdras": 10,
"Neemias": 13,
"Ester": 10,
"Jó": 42,
"Salmos": 150,
"Provérbios": 31,
"Eclesiastes": 12,
"Cântico%20dos%20Cânticos": 8,
"Isaías": 66,
"Jeremias": 52,
"Lamentações%20de%20Jeremias": 5,
"Ezequiel": 48,
"Daniel": 12,
"Oseias": 14,
"Joel": 3,
"Amós": 9,
"Obadias": 1,
"Jonas": 4,
"Miqueias": 7,
"Naum": 3,
"Habacuque": 3,
"Sofonias": 3,
"Ageu": 2,
"Zacarias":14,
"Malaquias": 4}

dfARCOld["livro"] = dfARCOld["livro"].replace("Cantares", "Cântico%20dos%20Cânticos")
dfARCOld["livro"] = dfARCOld["livro"].replace("Lamentações", "Lamentações%20de%20Jeremias")

dfNVIPTOld["livro"] = dfNVIPTOld["livro"].replace("Miquéias", "Miqueias")
dfNVIPTOld["livro"] = dfNVIPTOld["livro"].replace("Cantares%20de%20Salomâo", "Cântico%20dos%20Cânticos")

dfNVTOld["livro"] = dfNVTOld["livro"].replace("Miquéias", "Miqueias")
dfNVTOld["livro"] = dfNVTOld["livro"].replace("Cantares%20de%20Salomâo", "Cântico%20dos%20Cânticos")

dfOLOld["livro"] = dfOLOld["livro"].replace("Miquéias", "Miqueias")
dfOLOld["livro"] = dfOLOld["livro"].replace("Cantares%20de%20Salomâo", "Cântico%20dos%20Cânticos")


print(set(dfNTLOld["livro"]) - set(dfARCOld["livro"]))
print(set(dfNTLOld["livro"]) - set(dfNVIPTOld["livro"]))
print(set(dfNTLOld["livro"]) - set(dfNVTOld["livro"]))
print(set(dfNTLOld["livro"]) - set(dfOLOld["livro"]))





set()
set()
set()
set()


In [26]:

import numpy as np
#Matriz do velho testamento

#         dfNTLOld   dfARCOld   dfVFLOld  dfNVIPTOld   dfNVTOld  dfOLOld
#dfNTLOld
  
#dfARCOld

#dfVFLOld

#dfNVIPTOld

#dfNVTOld

#dfOLOld



[[    1 21691     1 22613 21684 22052]
 [    0     1     1 22657 22656 22021]
 [    0     0     1     1     1     1]
 [    0     0     0     1 22832 23030]
 [    0     0     0     0     1 22310]
 [    0     0     0     0     0     1]]


In [28]:

#Matriz do novo testamento

#         dfNTLNew   dfARCNew   dfVFLNew  dfNVIPTNew   dfNVTNew  dfOLNew
#dfNTLNew  

#dfARCNew

#dfVFLNew

#dfNVIPTNew

#dfNVTNew

#dfOLNew

[[   1 7638 7616 7896 7639 7835]
 [   1    1 7644 7686 7687 7627]
 [   1    1    1 7648 7651 7593]
 [   1    1    1    1 7690 7880]
 [   1    1    1    1    1 7634]
 [   1    1    1    1    1    1]]


In [10]:
dfNTLNew.to_csv("./data/NTLHNew.tsv", sep="\t")
dfNTLOld.to_csv("./data/NTLHOld.tsv", sep="\t")

dfARCNew.to_csv("./data/ARCNew.tsv", sep="\t")
dfARCOld.to_csv("./data/ARCOld.tsv", sep="\t")

dfNVIPTNew.to_csv("./data/NVIPTNew.tsv", sep="\t")
dfNVIPTOld.to_csv("./data/NVIPTOld.tsv", sep="\t")

dfNVTNew.to_csv("./data/NVTNew.tsv", sep="\t")
dfNVTOld.to_csv("./data/NVTOld.tsv", sep="\t")

dfOLNew.to_csv("./data/OLNew.tsv", sep="\t")
dfOLOld.to_csv("./data/OLOld.tsv", sep="\t")

dfVFLNew.to_csv("./data/VFLNew.tsv", sep="\t")

In [9]:
from nltk import ngrams
from sacremoses import MosesTokenizer

def tokenize(text, nGram=3, lang="pt"):
    entok = MosesTokenizer(lang=lang)
    text = entok.tokenize(text, escape=False)
    grams = []
    for i in range(1, nGram):
        i_grams = [
            " ".join(gram)
            for gram in ngrams(text, i)
        ]
        grams.extend(i_grams)
        
    return grams

def getNgramOverlap(hypothesys, references, nGram):

  overlaps = []
  for h, r in zip([hypothesys], [references]):
    if (h == "") or (r == ""):
      overlaps.append(1.0)
      continue
    a = tokenize(h, nGram)
    b = tokenize(r, nGram)

    if len(a) >= len(b):
      overlaps.append(len(set(a) & set(b))/len(a))
    elif len(b) >= len(a):
      overlaps.append(len(set(a) & set(b))/len(b))

  return overlaps[0]


In [4]:
import numpy as np

def getStats(dfA, dfB):
    df = dfA.merge(dfB, on=["livro", "capitulo", "versiculo"])

    df["sourceLen"] = df["texto_x"].apply(lambda x: getSizeSentece(x))
    df["targetLen"] = df["texto_y"].apply(lambda x: getSizeSentece(x))
    
    df["overlap"] = df.apply(lambda x: getNgramOverlap(x["texto_x"], x["texto_y"], 3), axis=1)

    return df
#Matriz do novo testamento

def getSizeSentece(text):
    try:
        return len(text.split(" "))
    except:
        return 0
    



In [16]:
print("Novo testamento: ")

#TODO


# df[df["overlap"] > 0.02] fazer esse corte
dfs = [getStats(dfNTLNew, dfARCNew), getStats(dfNTLNew, dfVFLNew), getStats(dfNTLNew, dfNVIPTNew), getStats(dfNTLNew, dfNVTNew), getStats(dfNTLNew, dfOLNew),
      getStats(dfARCNew, dfVFLNew), getStats(dfARCNew, dfNVIPTNew), getStats(dfARCNew, dfNVTNew), getStats(dfARCNew, dfOLNew),
       getStats(dfVFLNew, dfNVIPTNew), getStats(dfVFLNew, dfNVTNew), getStats(dfVFLNew, dfOLNew),
       getStats(dfNVIPTNew, dfNVTNew), getStats(dfNVIPTNew, dfOLNew),
       getStats(dfNVTNew, dfOLNew)]


dfConcat = pd.concat(dfs, ignore_index=False)
#remove versiculos deseconhecidos
dfConcat = dfConcat[dfConcat["versiculo"] <= 180]
#remove textos muito difenretes
dfConcat = dfConcat[dfConcat["overlap"] > 0.02]

dfConcat = dfConcat[dfConcat["sourceLen"] > 5]
dfConcat = dfConcat[dfConcat["targetLen"] > 5]

print("tamanho sem cortes: ", dfConcat.shape)
dfConcat.drop(["Unnamed: 0_x", "Unnamed: 0_y"], axis=1, inplace=True)

dfConcat.to_csv("./data/PT/NovoTestamentoCompleto-PTBR.tsv", sep="\t", index=False)

Novo testamento: 
tamanho sem cortes:  (113584, 12)


In [48]:
dfConcat.sample(20).drop(["Unnamed: 0_x", "Unnamed: 0_y"],axis=1)

,estilo_x,livro,capitulo,versiculo,texto_x,estilo_y,texto_y,sourceLen,targetLen,overlap
2049,NVT,Lucas,8,18,"“ Portanto , ouçam com atenção ! Pois ao que t...",OL,"Tomem cuidado como ouvem , pois quem tiver rec...",38,37,0.253333
794,NVI-PT,Mateus,23,24,Guias cegos ! Vocês coam um mosquito e engolem...,OL,"Guias cegos ! Coam um mosquito , mas seriam ca...",12,15,0.344828
5490,VFL,2%20Coríntios,4,2,"Pelo contrário , deixamos de fazer as coisas v...",NVT,Rejeitamos todos os atos vergonhosos e métodos...,60,39,0.151261
2292,ARC,Lucas,12,32,"Não temas , ó pequeno rebanho , porque a vosso...",VFL,"Não tenha medo , meu pequeno rebanho , pois o ...",16,20,0.230769
1688,NTLH,Marcos,16,6,mas ele disse : — Não se assustem ! Sei que vo...,NVT,mas ele disse : “ Não tenham medo . Vocês proc...,41,41,0.407407
5961,NVI-PT,Efésios,5,21,"Sujeitem-se uns aos outros , por temor a Crist...",NVT,Sujeitem-se uns aos outros por temor a Cristo .,12,9,0.695652
2868,NTLH,João,1,6,"Houve um homem chamado João , que foi enviado ...",OL,"Apareceu um homem , enviado por Deus , chamado...",11,11,0.571429
2363,VFL,Lucas,14,31,Ou se um rei quiser entrar numa batalha contra...,OL,E qual é o rei que se dispõe a iniciar uma gue...,39,22,0.103896
4582,NTLH,Romanos,1,7,Por isso eu escrevo a todos vocês que estão em...,NVT,"Escrevo a todos vocês que estão em Roma , amad...",55,41,0.366972
6602,NTLH,Hebreus,4,12,Pois a palavra de Deus é viva e poderosa e cor...,ARC,"Porque a palavra de Deus é viva , e eficaz , e...",50,52,0.349515


In [15]:

print("Velho testamento: ")

#         dfNTLOld   dfARCOld   dfVFLOld  dfNVIPTOld   dfNVTOld  dfOLOld
#dfNTLOld
  
#dfARCOld

#dfVFLOld

#dfNVIPTOld

#dfNVTOld

#dfOLOld


dfs = [getStats(dfNTLOld, dfARCOld), getStats(dfNTLOld, dfNVIPTOld), getStats(dfNTLOld, dfNVTOld), getStats(dfNTLOld, dfOLOld),
      getStats(dfARCOld, dfNVIPTOld), getStats(dfARCOld, dfNVTOld), getStats(dfARCOld, dfOLOld),
      getStats(dfNVIPTOld, dfNVTOld), getStats(dfNVIPTOld, dfOLOld),
      getStats(dfNVTOld, dfOLOld)]



dfConcat = pd.concat(dfs, ignore_index=False)
#remove versiculos deseconhecidos
dfConcat = dfConcat[dfConcat["versiculo"] <= 180]
#remove textos muito difenretes
dfConcat = dfConcat[dfConcat["overlap"] > 0.02]
#filtra sentencas curtas
dfConcat = dfConcat[dfConcat["sourceLen"] > 5]
dfConcat = dfConcat[dfConcat["targetLen"] > 5]

print("tamanho sem cortes: ", dfConcat.shape)
dfConcat.drop(["Unnamed: 0_x", "Unnamed: 0_y"], axis=1, inplace=True)
dfConcat.to_csv("./data/PT/VelhoTestamentoCompleto-PTBR.tsv", sep="\t", index=False)

Velho testamento: 
tamanho sem cortes:  (214611, 12)


In [18]:
dfConcat.sample(20)

,estilo_x,livro,capitulo,versiculo,texto_x,estilo_y,texto_y,sourceLen,targetLen,overlap
3950,ARC,Atos,8,11,"E atendiam-no a ele , porque já desde muito te...",NVI-PT,"Eles o seguiam , pois ele os havia iludido com...",17,16,0.393939
891,VFL,Mateus,26,6,"Jesus estava na cidade de Betânia , na casa de...",NVT,"Enquanto isso , Jesus estava em Betânia , na c...",15,16,0.612903
1981,ARC,Lucas,6,48,"É semelhante ao homem que edificou uma casa , ...",OL,são como o homem que constrói uma casa sobre a...,49,33,0.144330
163,NTLH,Mateus,6,31,"Portanto , não fiquem preocupados , perguntand...",VFL,"—Portanto , não fiquem preocupados , dizendo :...",37,34,0.273973
731,VFL,Mateus,22,25,"Ora , entre nós havia sete irmãos . O primeiro...",OL,Houve entre nós uma família de sete irmãos . O...,34,28,0.313433
1619,NTLH,Marcos,14,53,"Em seguida , levaram Jesus até a casa do Grand...",ARC,"E levaram Jesus ao sumo sacerdote , e ajuntara...",30,23,0.186441
1716,NTLH,Marcos,15,36,Alguém correu e molhou uma esponja em vinho co...,NVI-PT,"Um deles correu , embebeu uma esponja em vinag...",36,37,0.356164
7647,NTLH,Apocalipse,9,17,Na minha visão vi os cavalos e os seus cavalei...,NVI-PT,Os cavalos e os cavaleiros que vi em minha vis...,50,55,0.440367
7436,ARC,Apocalipse,9,9,E tinham couraças como couraças de ferro ; e o...,NVI-PT,"Tinham couraças como couraças de ferro , e o s...",28,27,0.454545
4350,NTLH,Atos,17,26,De um só homem ele criou todas as raças humana...,OL,"Criou toda a população do mundo , a partir de ...",36,39,0.142857


In [ ]:

dfConcat


Quem vive pensando e dizendo coisas más não pode esperar nada de bom , mas só a desgraça .
O perverso de coração nunca achará o bem ; e o que tem a língua dobre virá a cair no mal .
